In [22]:
import numpy as np # used to perform a wide variety of mathematical operations on arrays
import pandas as pd # used to perform data manipulation and analysis

In [3]:
df= pd.read_csv("/Users/isslamkhatir/Desktop/Projets superset/clean_data/retail_clean.csv", parse_dates=["InvoiceDate"])
df.head(5)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Hour,Date,Weekday,Month,Week,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,7,2009-12-01,Tuesday,December,49,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,7,2009-12-01,Tuesday,December,49,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,7,2009-12-01,Tuesday,December,49,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,7,2009-12-01,Tuesday,December,49,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,7,2009-12-01,Tuesday,December,49,30.0


In [4]:
df.isna().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
Hour           0
Date           0
Weekday        0
Month          0
Week           0
TotalPrice     0
dtype: int64

In [6]:
# Deletion of lines with negative or zero quantities or prices
df = df[(df["Quantity"] > 0) & (df["Price"] > 0)]
df.describe()

,Quantity,Price,Customer ID,Hour,Week,TotalPrice
count,407664.000000,407664.000000,407664.000000,407664.000000,407664.000000,407664.000000
mean,13.585585,3.294438,15368.592598,12.873962,30.024081,21.664909
std,96.840747,34.757965,1679.762138,2.303916,15.045607,77.150058
min,1.000000,0.001000,12346.000000,7.000000,1.000000,0.000000
25%,2.000000,1.250000,13997.000000,11.000000,17.000000,4.950000
50%,5.000000,1.950000,15321.000000,13.000000,32.000000,11.900000
75%,12.000000,3.750000,16812.000000,14.000000,44.000000,19.500000
max,19152.000000,10953.500000,18287.000000,20.000000,52.000000,15818.400000


# Calcul RFM (Recency, Frequency, Monetary)

### Calculate the RFM indicators for each client:

**Recency:** Number of days since the last purchase

**Frequency:** Number of purchases over the analyzed period

**Monetary:** Total amount spent over the period

In [8]:
# Reference date for the recency (ex: one day after the last order)
ref_date = df["InvoiceDate"].max() + pd.Timedelta(days=1)
ref_date

Timestamp('2010-12-10 20:01:00')

In [16]:
# RFM par client
rfm = df.groupby("Customer ID").agg({
    "InvoiceDate": lambda x: (ref_date - x.max()).days,  # Recency
    "Invoice": "nunique",                                # Frequency
    "TotalPrice": "sum"                                  # Monetary
}).rename(columns={
    "InvoiceDate": "Recency",
    "Invoice": "Frequency",
    "TotalPrice": "Monetary"
})

rfm.head(5)

,Recency,Frequency,Monetary
Customer ID,,,
12346.0,165,11,372.86
12347.0,3,2,1323.32
12348.0,74,1,222.16
12349.0,43,3,2671.14
12351.0,11,1,300.93


# Segmentation by RFM score

In [21]:
rfm["R_score"] = pd.qcut(rfm["Recency"], q=4, labels=[4,3,2,1])
rfm["F_score"] = pd.qcut(rfm["Frequency"].rank(method="first"), q=4, labels=[1,2,3,4])
rfm["M_score"] = pd.qcut(rfm["Monetary"], q=4, labels=[1,2,3,4])
rfm["RFM_Score"] = rfm["R_score"].astype(str) + rfm["F_score"].astype(str) + rfm["M_score"].astype(str)
rfm["RFM_Score"].head(10).reset_index()

,Customer ID,RFM_Score
0,12346.0,142
1,12347.0,423
2,12348.0,211
3,12349.0,334
4,12351.0,411
5,12352.0,422
6,12353.0,312
7,12355.0,112
8,12356.0,434
9,12357.0,324


In [23]:
rfm.to_csv("/Users/isslamkhatir/Desktop/Projets superset/clean_data/rfm_scores1.csv", index=False)